In [ ]:
# Import necessary libraries
import os
import shutil


# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')


# Install and Setup PostgreSQL
!apt-get -y -qq install postgresql postgresql-contrib &> /dev/null
!service postgresql start
!sudo -u postgres createuser --superuser adsdb
!sudo -u postgres createdb adsdb
!sudo -u postgres psql -c "ALTER USER adsdb WITH PASSWORD 'adsdb';"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE


In [ ]:
drive_path = "/content/drive/MyDrive/ADSDB/trusted/dumpfile.sql"
dump_path= "/content/dumpfile.sql"
# Check if the dumpfile exists in the Google Drive
if not os.path.exists(drive_path):
  print("Dump file not found in Google Drive.")


# Copy the dumpfile from Google Drive to Colab's environment
shutil.copy(drive_path, dump_path)
# Create a temporary .pgpass file for authentication
with open("/root/.pgpass", "w") as f:
  f.write("*:*:*:adsdb:adsdb")
os.chmod("/root/.pgpass", 0o600)  # Set the required permissions

# Restore the database using psql command
try:
  !PGPASSFILE=/root/.pgpass psql -h localhost -U adsdb -d adsdb < "{dump_path}"
  print("Restoration successful!")
except Exception as e:
  print(f"Error during restoration: {e}")

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 28620
COPY 14400
COPY 900
COPY 15
Restoration successful!


In [ ]:
!PGPASSFILE=/root/.pgpass psql -h localhost -U adsdb -d adsdb -c "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"

           table_name            
---------------------------------
 final_employment_rate
 final_house_price_index
 final_house_price_index_weights
 final_inflation_rate
(4 rows)



In [ ]:
table_names = ['final_employment_rate', 'final_house_price_index_weights', 'final_inflation_rate', 'final_house_price_index']

def show_table(table_name):
  display_query = f"SELECT * FROM {table_name} LIMIT 20;"
  !PGPASSFILE=/root/.pgpass psql -h localhost -U adsdb -d adsdb -c "{display_query}"

def check_for_duplicates(table_name):
    if 'inflation' in table_name:
      duplicate_query = f"""
      SELECT year, count(*)
      FROM {table_name}
      GROUP BY year
      HAVING COUNT(*) > 1
      LIMIT 20;
      """
    elif 'employment' in table_name:
      duplicate_query = f"""
      SELECT year, quarter, provinces, rates, sex, count(*)
      FROM {table_name}
      GROUP BY year, quarter, provinces, rates, sex
      HAVING COUNT(*) > 1
      ORDER BY year, quarter
      LIMIT 20;
      """
    elif 'weights' in table_name:
      duplicate_query = f"""
      SELECT year, autonomous_communities_and_cities , index_type, count(*)
      FROM {table_name}
      GROUP BY year, autonomous_communities_and_cities , index_type
      HAVING COUNT(*) > 1
      LIMIT 20;
      """
    elif (table_name == 'final_house_price_index'):
      duplicate_query = f"""
      SELECT year, quarter, indices_and_rates, index_type, autonomous_communities_and_cities, national_total, count(*)
      FROM {table_name}
      GROUP BY year, quarter, indices_and_rates, index_type, autonomous_communities_and_cities, national_total
      HAVING COUNT(*) > 1
      LIMIT 20;
      """
    !PGPASSFILE=/root/.pgpass psql -h localhost -U adsdb -d adsdb -c "{duplicate_query}"

for table_name in table_names:
  #get_primary_keys(table_name)
  check_for_duplicates(table_name)

 year | quarter | provinces | rates | sex | count 
------+---------+-----------+-------+-----+-------
(0 rows)

 year | autonomous_communities_and_cities | index_type | count 
------+-----------------------------------+------------+-------
(0 rows)

 year | count 
------+-------
(0 rows)

 year | quarter | indices_and_rates | index_type | autonomous_communities_and_cities | national_total | count 
------+---------+-------------------+------------+-----------------------------------+----------------+-------
(0 rows)



In [ ]:
show_table('final_inflation_rate')
show_table('final_employment_rate')


 year | inflation_rate_percent | annual_change 
------+------------------------+---------------
 2008 |                   4.08 |          1.29
 2009 |                  -0.29 |         -4.36
 2010 |                   1.80 |          2.09
 2011 |                   3.20 |          1.40
 2012 |                   2.45 |         -0.75
 2013 |                   1.41 |         -1.04
 2014 |                  -0.15 |         -1.56
 2015 |                  -0.50 |         -0.35
 2016 |                  -0.20 |          0.30
 2017 |                   1.96 |          2.16
 2018 |                   1.68 |         -0.28
 2019 |                   0.70 |         -0.98
 2020 |                  -0.32 |         -1.02
 2021 |                   3.09 |          3.42
 2022 |                   8.39 |          5.30
(15 rows)

 id | year |    sex     |   provinces    |       rates       | total | quarter 
----+------+------------+----------------+-------------------+-------+---------
  1 | 2008 | Both sexes | Na